In [1]:
def qZ(x, y):
    return (x - 3 * y + 1) / (3 * x ** 2 + 3 * y ** 2 + 1)


def qSumZ(z):
    return sum(z)


def exchange_scheme(oldX, oldY, sortedId):
    x = [0 for i in range(4)]
    y = [0 for i in range(4)]
    x[0] = oldX[sortedId[0]]
    x[1] = oldX[sortedId[1]]
    x[2] = oldX[sortedId[2]]
    x[3] = oldX[sortedId[2]]

    y[0] = oldY[sortedId[2]]
    y[1] = oldY[sortedId[2]]
    y[2] = oldY[sortedId[0]]
    y[3] = oldY[sortedId[1]]
    return x, y


def sorting(z):
    sortedId = sorted(range(len(z)), key=lambda k: z[k])
    return sortedId


def evoStep(x, y, z):
    _, minId = min((value, id) for (id, value) in enumerate(z))
    x = x[:]
    y = y[:]
    z = z[:]
    x.pop(minId)
    y.pop(minId)
    z.pop(minId)
    return x, y, z


def evoSteps(x, y, stepsNum=4):
    results = []
    for i in range(4):
        arrZ = [qZ(x, y[i]) for i, x in enumerate(x)]
        x, y, z = evoStep(x, y, arrZ)
        x, y = exchange_scheme(x, y, sorting(z))
        results.append([x, y, qSumZ(arrZ), arrZ])
    return x, y, results


x = [-2, -1, 0, 1]
y = [-2, -1, 0, 1]
results = evoSteps(x, y)
for i in range(len(results[2])):
    print(f'max_{i + 1}_step: {results[2][i][2]}')
qualityArrZ = []
for i in range(len(results[2])):
    qualityArrZ += results[2][i][3]
print(f'max Z: {max(qualityArrZ)}')


max_1_step: 1.4857142857142858
max_2_step: 1.4615384615384615
max_3_step: 2.967032967032967
max_4_step: 3.5384615384615383
max Z: 1.0


In [9]:
import networkx as nx
from math import e


def probability(delta, T):
    return 100 * e ** (-delta / T)


def reductTemp(prevT):
    nextT = 0.5 * prevT
    return nextT


def edgeLength(i, j, distances, roundTrip=True):
    if roundTrip:
        return max([(item[2] if (item[0] == i and item[1] == j) or (item[1] == i and item[0] == j) else -1) for item in
                    distances])
    else:
        return max([(item[2] if (item[0] == i and item[1] == j) else -1) for item in distances])


def routeLength(V, distances):
    edges = []
    for i in range(len(V) - 1):
        edges.append(edgeLength(V[i], V[i + 1], distances))
    return sum(edges)


def routeOneReplacement(arrV, Z, replacementByName=True):
    decrement = 1 if replacementByName else 0
    arrV[Z[0] - decrement], arrV[Z[1] - decrement] = arrV[Z[1] - decrement], arrV[Z[0] - decrement]
    return arrV


def routeReplacement(V, Z):
    for z in Z:
        V = routeOneReplacement(V, z)
    return V


def chooseRoute(distances, V, Z, T, P):
    sumLength = routeLength(V, distances)
    arrSum = [sumLength]
    for i in range(len(Z)):
        newV = routeOneReplacement(V[:], Z[i])
        newS = routeLength(newV, distances)
        arrSum.append(newS)
        deltaS = newS - sumLength
        if deltaS > 0:
            p = probability(deltaS, T)
            if p > P[i]:
                V = newV
                sumLength = newS
        else:
            V = newV
            sumLength = newS
        T = reductTemp(T)
    return V, arrSum


def drawRouteGraph(distances, bestRoute):
    newDistances = []
    for i in range(len(bestRoute) - 1):
        for distance in distances:
            if distance[0] == bestRoute[i] and distance[1] == bestRoute[i + 1] or distance[1] == bestRoute[i] and \
                    distance[0] == bestRoute[i + 1]:
                newDistances.append(distance)
    graph = nx.Graph()
    graph.add_weighted_edges_from(newDistances)
    nx.draw_kamada_kawai(graph, node_color='#fb7258', node_size=2000, with_labels=True)


distances = [(1, 2, 26), (1, 3, 42), (1, 4, 44), (1, 5, 31), (1, 6, 24),
             (2, 3, 20), (2, 4, 34), (2, 5, 40), (2, 6, 15),
             (3, 4, 23), (3, 5, 43), (3, 6, 20),
             (4, 5, 27), (4, 6, 22),
             (5, 6, 26)]
V = [1, 2, 3, 4, 5, 6, 1]
Z = [(3, 4), (4, 6), (5, 6), (6, 2)]
P = [90, 45, 43, 31]
T = 100
graph = nx.Graph()
graph.add_weighted_edges_from(distances)

# Use the line below to plot full graph
# nx.draw_kamada_kawai(graph, node_color='#fb7258', node_size=2000, with_labels=True)

bestRoute, arrLength = chooseRoute(distances, V, Z, T, P)
print(f'Best route: {bestRoute}')
print(f'Best route length: {routeLength(bestRoute, distances)}')
print(f'All routes lengths: {arrLength}')

# Use the line below to plot the bestRoute
# drawRouteGraph(distances, bestRoute)


Best route: [1, 2, 3, 6, 4, 5, 1]
Best route length: 146
All routes lengths: [146, 176, 163, 146, 176]
